In [19]:
import gzip
import json
from pathlib import Path
import rdflib
import subprocess

In [4]:
in_file_1 = '/mnt/dane/work/rdf_streaming/sripas_weather/weather.json.gz'
in_file_2 = '/mnt/dane/work/rdf_streaming/sripas_weather/data.json.gz'
out_dir = Path('../data/out/sripas-weather')

In [23]:
num = 0
dir_levels = 1

def do_file(buf: str):
    with open('temp/data.json', 'wt') as fp:
        fp.write(buf)
    s_res = subprocess.run(['java', '-jar', 'bin/mapper.jar'])
    if s_res.returncode != 0:
        print(s_res.stderr.decode('utf-8'))
        return
    g = rdflib.Graph()
    g.parse(data=s_res.stdout.decode('utf-8'))


with gzip.open(in_file_1, 'rt') as f:
    for line in f:
        do_file(line)
        break

with gzip.open(in_file_2, 'rt') as f:
    buffer = ''
    for line in f:
        buffer += line
        if line.startswith('}'):
            data = json.loads(buffer)['payload']
            do_file(json.dumps(data))
            buffer = ''
            break

18:16:52.206 [main] ERROR be.ugent.rml.cli.Main               .run(298) - Failed to make mapping file conformant to RML spec.
java.lang.Exception: Mapping requires at least one TriplesMap
	at be.ugent.rml.conformer.MappingConformer.detect(MappingConformer.java:80)
	at be.ugent.rml.conformer.MappingConformer.conform(MappingConformer.java:55)
	at be.ugent.rml.cli.Main.run(Main.java:292)
	at be.ugent.rml.cli.Main.main(Main.java:46)
Exception in thread "main" java.lang.Error: No Triples Maps found. The mapping document you should at least have one Triples Map.
	at be.ugent.rml.Initializer.getAllTriplesMaps(Initializer.java:59)
	at be.ugent.rml.Initializer.<init>(Initializer.java:23)
	at be.ugent.rml.Executor.<init>(Executor.java:52)
	at be.ugent.rml.cli.Main.run(Main.java:390)
	at be.ugent.rml.cli.Main.main(Main.java:46)


AttributeError: 'NoneType' object has no attribute 'decode'